In [89]:
# duh
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# scikit
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
# evaluate data
from sklearn.model_selection import cross_val_score
from scipy.stats.stats import pearsonr

In [90]:
# load csv
train = pd.read_csv("./data/train.csv", dtype={"Age": np.float64}, )
test = pd.read_csv("./data/test.csv", dtype={"Age": np.float64}, )

train.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


In [91]:
# handle missing age and embarked
train.Age.fillna(train.Age.mean(), inplace=True)
train.Embarked.fillna('S', inplace=True)

In [92]:
le = preprocessing.LabelEncoder()
le.fit(pd.unique(train.Sex))
#le.classes_ for printing distinct classes
sex_t = le.transform(train.Sex)
test_sex_t = le.transform(test.Sex)

#label encoding for embarked
le = preprocessing.LabelEncoder()
le.fit(pd.unique(train.Embarked))
embarked_t = le.transform(train.Embarked)
test_embarked_t = le.transform(test.Embarked)

In [93]:
Y = train.iloc[:,1]
train = train.iloc[:,[2,4,5,6,7,9,11]]
test = test.iloc[:,[1,3,4,5,6,8,10]]


train.iloc[:,1] = sex_t
test.iloc[:,1] = test_sex_t

train.iloc[:,6] = embarked_t
test.iloc[:,6] = test_embarked_t

print(train.head(3))
print(test.head(3))

   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0       3    1  22.0      1      0   7.2500         2
1       1    0  38.0      1      0  71.2833         0
2       3    0  26.0      0      0   7.9250         2
   Pclass  Sex   Age  SibSp  Parch    Fare  Embarked
0       3    1  34.5      0      0  7.8292         1
1       3    0  47.0      1      0  7.0000         2
2       2    1  62.0      0      0  9.6875         1


In [94]:
# normalize
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

scaler = StandardScaler()
scaler.fit(train.values)

X_train = scaler.transform(train)
X_test = scaler.transform(test)

In [95]:
pclassco = pearsonr(Y, X_train[:,0])
sexco = pearsonr(Y, X_train[:,1])
ageco = pearsonr(Y, X_train[:,2])
sibspco = pearsonr(Y, X_train[:,3])
parchco = pearsonr(Y, X_train[:,4])
embarkedco = pearsonr(Y, X_train[:,5])
family = pearsonr(Y,X_train[:,3] + X_train[:,4])
print(pclassco)
print(sexco)
print(ageco)
print(sibspco)
print(parchco)
print(embarkedco)
print(family)

(-0.33848103596101525, 2.537047387978631e-25)
(-0.5433513806577551, 1.406066130878875e-69)
(-0.06980851528714312, 0.03721708372681355)
(-0.03532249888573556, 0.2922439286980196)
(0.08162940708348347, 0.014799245374712841)
(0.25730652238496243, 6.120189341917992e-15)
(0.027528178502442976, 0.41181275255288907)


In [81]:
# grid search w/ SVM
svm = SVC()

clf = GridSearchCV(svm, {'kernel':('linear', 'rbf'), 'C':(1,0.25,0.5,0.75),'gamma': (1,2,3,'auto'),'decision_function_shape':('ovo','ovr'),'shrinking':(True,False)})
clf.fit(X_train,Y)

print("accuracy:"+str(np.average(cross_val_score(clf, X_train, Y, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(clf, X_train, Y, scoring='f1'))))

accuracy:0.827154604230745
f1:0.7545126842698361


In [83]:
clf.best_params_

{'C': 1,
 'decision_function_shape': 'ovo',
 'gamma': 'auto',
 'kernel': 'rbf',
 'shrinking': True}

In [88]:
from sklearn.pipeline import Pipeline

def plot_classifier_graph(X, y, clf):
  plt.figure(figsize=(10,10))
  plot_decision_regions(X, y, clf=clf, legend=2)
  plt.legend()
  plt.tight_layout()

poly_kernel_svm_clf = Pipeline([
                               ("scaler", StandardScaler()),
                               ("svm_clf", SVC(kernel="rbf", gamma='auto', C=1)),
])
poly_kernel_svm_clf.fit(X_train, Y)
plot_classifier_graph(X_train, Y, poly_kernel_svm_clf)

output = clf.predict(test).astype(int)
df_output = pd.DataFrame()
aux = pd.read_csv('./data/test.csv')
df_output['PassengerId'] = aux['PassengerId']
df_output['Survived'] = output
df_output[['PassengerId','Survived']].to_csv('predictions.csv', index=False)

NameError: name 'plt' is not defined